In [1]:
import pandas as pd
import numpy as np
import pyodbc
import math
from ast import literal_eval
from tqdm import tqdm
import os
from configparser import ConfigParser
import statistics
from scipy import stats as s

# from connector.mosaicio import MosaicioConnector

In [2]:
# data_ind_U  = pd.read_csv(r"/notebooks/notebooks/Synthetic_data_Clustered_individual_U.csv")
# #uncomment to run for org
# # data_org_U = pd.read_csv(r"C:\Users\FIU\Desktop\EntityRisk_DEMO\Synthetic_data_Clustered_organization_U.csv")

# ind_data_thresholds_U = pd.read_csv(r"/notebooks/notebooks/threshold_individual_U.csv")
# #uncomment to run for org
# # org_data_thresholds_U = pd.read_csv(r"C:\Users\FIU\Desktop\EntityRisk_DEMO\threshold_organization_U.csv")

# weight_table_ind_U = pd.read_csv(r"/notebooks/notebooks/weight_individual_U.csv")
# #uncomment to run for org
# # weight_table_org_U = pd.read_csv(r"C:\Users\FIU\Desktop\EntityRisk_DEMO\weight_organization_U.csv")

In [3]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [ ]:
import math
def round_func(num):
    if num-math.floor(num)<0.5:
        return math.floor(num)
    return math.ceil(num)

In [4]:
cursor.execute('''INSERT INTO  FINCORE_DB.Fincore_Analytics.RISK_PERSON_ANALYTICAL_RULES
select
A.PERSON_MASTER_ID,
A.RULE_NO,
case 
	when c.THRESHOLD1_1 IS null then 0
	when A.RULE_VALUE < c.threshold1_1 then 0 
	when A.RULE_VALUE = c.threshold1_1 then 1
	when A.RULE_VALUE <= c.THRESHOLD1_2 then 2
	when A.RULE_VALUE <= c.THRESHOLD1_3 then 3
	when A.RULE_VALUE <= c.THRESHOLD1_4 then 4
	when A.RULE_VALUE <= c.THRESHOLD1_5 then 5
	when A.RULE_VALUE <= c.THRESHOLD1_6 then 6
	when A.RULE_VALUE <= c.THRESHOLD1_7 then 7
	when A.RULE_VALUE <= c.THRESHOLD1_8 then 8
	when A.RULE_VALUE <= c.THRESHOLD1_9 then 9
	when A.RULE_VALUE <= c.THRESHOLD1_10 then 10
	else 10
	end Risk_score,
	GETDATE() AS CREATE_DATE
from FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG_rules a 
inner join FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG b 
on a.PERSON_MASTER_ID = b.PERSON_MASTER_ID
inner join FINCORE_DB.Fincore_Analytics.RISK_lkp_entity_threshold c on 
c.PEER_GROUP_ID = b.PEER_GROUP_ID and a.RULE_NO = c.RULE_NO
where a.RISK_SCORE_STATIC is null''')
conn.commit()

In [23]:
risk_an_per = pd.read_sql_query('''select 
A.PERSON_MASTER_ID,
A.RULE_NO,
A.RULE_VALUE,
case 
	when c.THRESHOLD1_1 IS null then 0
	when A.RULE_VALUE < c.threshold1_1 then 0 
	when A.RULE_VALUE = c.threshold1_1 then 1
	when A.RULE_VALUE <= c.THRESHOLD1_2 then 2
	when A.RULE_VALUE <= c.THRESHOLD1_3 then 3
	when A.RULE_VALUE <= c.THRESHOLD1_4 then 4
	when A.RULE_VALUE <= c.THRESHOLD1_5 then 5
	when A.RULE_VALUE <= c.THRESHOLD1_6 then 6
	when A.RULE_VALUE <= c.THRESHOLD1_7 then 7
	when A.RULE_VALUE <= c.THRESHOLD1_8 then 8
	when A.RULE_VALUE <= c.THRESHOLD1_9 then 9
	when A.RULE_VALUE <= c.THRESHOLD1_10 then 10
	else 10
	end THRESHOLD_FLG,
	GETDATE() AS CREATE_DATE
from FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG_rules a 
inner join FINCORE_DB.Fincore_Analytics.RISK_PERSON_SUMMARY_STG b 
on a.PERSON_MASTER_ID = b.PERSON_MASTER_ID
inner join FINCORE_DB.Fincore_Analytics.RISK_lkp_entity_threshold c on 
c.PEER_GROUP_ID = b.PEER_GROUP_ID and a.RULE_NO = c.RULE_NO
where a.RISK_SCORE_STATIC is null''',conn)

In [24]:
# for i in tqdm(range(10)):
#     df_cal = pd.read_csv((r"cal_") + str(i) + ".csv")
#     risk_an_per = risk_an_per.append(df_cal, ignore_index=True)

In [25]:
risk_an_per.drop_duplicates(inplace=True)
risk_an_per.to_csv(r"/data/risk_an_per.csv")

In [26]:
risk_an_per = pd.read_csv(r"/data/risk_an_per.csv")
risk_an_per.drop(columns = ['Unnamed: 0'], inplace = True)
risk_an_per.dropna(inplace =  True)

In [27]:
# config = ConfigParser()
# config.read(r'/notebooks/notebooks/config.ini')
# connection_string = config.get('SERVER', 'connection')
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()

In [28]:
risk_an_per.drop_duplicates(inplace=True)

In [29]:
risk_an_per

PERSON_MASTER_ID     RULE_NO  RULE_VALUE  THRESHOLD_FLG  \
0                   1  P002001007           1              1   
1                   1  P002001024           1              1   
2                   1  P002001025           1              1   
3                   2  P002001007           1              1   
4                   2  P002001024           1              1   
..                ...         ...         ...            ...   
889               228  P002001007           1              1   
890               228  P002001008           1              1   
891               228  P002001009           1              1   
892               228  P002001010           1              1   
893               228  P002001012           1              1   

                 CREATE_DATE  
0    2023-03-04 12:39:50.813  
1    2023-03-04 12:39:50.813  
2    2023-03-04 12:39:50.813  
3    2023-03-04 12:39:50.813  
4    2023-03-04 12:39:50.813  
..                       ...  
889  2023-03-04 12:39:50.813  
890  2023-03-04 12:39:50.813  
891  2023-03-04 12:39:50.813  
892  2023-03-04 12:39:50.813  
893  2023-03-04 12:39:50.813  

[894 rows x 5 columns]

In [30]:
# for ind,row in tqdm(risk_an_per.iterrows()):
#     conn.execute('''INSERT INTO [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_ANALYTICAL_RULES] (PERSON_MASTER_ID, RULE_NO, THRESHOLD_FLG)
#     values (?,?,?)''',(row['PERSON_MASTER_ID'],row['RULE_NO'],row['RISK_SCORE']))
# #     conn.execute('''INSERT INTO [Fincore_Analytics].[RISK_PERSON_ANALYTICAL_RULES] (PERSON_MASTER_ID, RULE_NO, RULE_VALUE,THRESHOLD_FLG) VALUES (?, ?, ?,?)''',(row['PERSON_MASTER_ID'],row['RULE_NO'],row['RULE_VALUE'],row['THRESHOLD_FLG']))
# #     conn.execute('INSERT INTO [Fincore_Analytics].[RISK_PERSON_ANALYTICAL_RULES] (PERSON_MASTER_ID, RULE_NO, RULE_VALUE,THRESHOLD_FLG) VALUES (?, ?, ?,?)', result)
# conn.commit()
# conn.close()


In [31]:
# config = ConfigParser()
# config.read(r'/notebooks/notebooks/config.ini')
# connection_string = config.get('SERVER', 'connection')
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()

In [32]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [33]:
risk_lkp = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_LKP_RULES] with (nolock)", conn)

In [34]:
# Make a list of all the Static rules

static_rules = risk_lkp[(risk_lkp['RISK_SCORE_STATIC_FLAG'] == 1) & (risk_lkp['IS_ACTIVE_FLAG'] == 1) & (risk_lkp['PERSON_TYPE'] == 'Individual') ]['RULE_NO'].to_list()
static_rules

['P002001013',
 'P003001001',
 'P003001002',
 'P003001004',
 'P003001007',
 'P003001008',
 'P003001009',
 'P003001010']

In [35]:
# Read risk_person_summary_stg_rules for the static rules
if len(static_rules) == 1:
    stat_rule_df = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES] with (nolock) where RULE_NO in ('{}')".format(static_rules[0]),conn)
    
elif(len(static_rules)== 0):
    stat_rule_df = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES] with (nolock) where RULE_NO in ('')",conn)
    
else:
    static_rules = tuple(static_rules)
    stat_rule_df = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES] with (nolock) where RULE_NO in" + str (static_rules) , conn)

# stat_rule_df = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES] where RULE_NO in "+str(tuple(static_rules)) , conn)


In [36]:
stat_rule_df[~pd.isnull(stat_rule_df['RISK_SCORE_STATIC'])]

PERSON_MASTER_ID     RULE_NO       RULE_VALUE  RISK_SCORE_STATIC  \
0                   1  P003001007       2022-12-29                 10   
1                   2  P003001007       2022-12-29                 10   
2                   3  P003001004  Savings Account                  6   
3                   3  P003001007       2022-12-29                 10   
4                   3  P003001009       2020-02-04                  8   
..                ...         ...              ...                ...   
257               227  P003001004  Savings Account                  6   
258               227  P003001009       2021-06-16                  9   
259               227  P003001010       2021-06-16                  9   
260               228  P003001009       2019-02-05                  7   
261               228  P003001010       2019-02-05                  7   

    CREATE_DATE  
0    2023-03-04  
1    2023-03-04  
2    2023-03-04  
3    2023-03-04  
4    2023-03-04  
..          ...  
257  2023-03-04  
258  2023-03-04  
259  2023-03-04  
260  2023-03-04  
261  2023-03-04  

[262 rows x 5 columns]

In [37]:
# The stat_rule_df is converted to a data frame which matches the shape of the risk_an_per

stat_rule_df = stat_rule_df[['PERSON_MASTER_ID','RULE_NO','RISK_SCORE_STATIC']]
stat_rule_df.drop_duplicates(inplace=True)

In [38]:
# stat_rule_df['THRESHOLD_FLG'] = 10
#Dont need to insert list based(static) rules in analytical table
stat_rule_df.rename(columns={'RISK_SCORE_STATIC':'RULE_VALUE'},inplace=True)
stat_rule_df['THRESHOLD_FLG'] = stat_rule_df['RULE_VALUE']

In [39]:
# Appending the data of static and non static rules to calculate the final risk score

risk_an_per = risk_an_per.append(stat_rule_df)

In [40]:
risk_an_per

PERSON_MASTER_ID     RULE_NO  RULE_VALUE  THRESHOLD_FLG  \
0                   1  P002001007           1              1   
1                   1  P002001024           1              1   
2                   1  P002001025           1              1   
3                   2  P002001007           1              1   
4                   2  P002001024           1              1   
..                ...         ...         ...            ...   
257               227  P003001004           6              6   
258               227  P003001009           9              9   
259               227  P003001010           9              9   
260               228  P003001009           7              7   
261               228  P003001010           7              7   

                 CREATE_DATE  
0    2023-03-04 12:39:50.813  
1    2023-03-04 12:39:50.813  
2    2023-03-04 12:39:50.813  
3    2023-03-04 12:39:50.813  
4    2023-03-04 12:39:50.813  
..                       ...  
257                      NaN  
258                      NaN  
259                      NaN  
260                      NaN  
261                      NaN  

[1154 rows x 5 columns]

In [41]:
temp = risk_an_per.groupby(['PERSON_MASTER_ID','RULE_NO']).size().reset_index()

In [42]:
risk_an_per.to_csv(r"/data/risk_an_per_with_static.csv")

In [43]:
# config = ConfigParser()
# config.read(r'/notebooks/notebooks/config.ini')
# connection_string = config.get('SERVER', 'connection')
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()

In [44]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [ ]:
error = pd.read_sql_query('''select PERSON_MASTER_ID,
rule_no,
COUNT(distinct risk_score_static)
from Fincore_Analytics.RISK_PERSON_SUMMARY_STG_RULES
group by PERSON_MASTER_ID,rule_no
having COUNT(distinct risk_score_static) > 1''',conn)
error

In [45]:
risk_per_analytical = risk_an_per
risk_weight =  pd.read_sql_query("SELECT * FROM FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES with (nolock) order by rule_no ",conn)

#Changing the structure of the table
risk_per_analytical = risk_per_analytical.pivot(index='PERSON_MASTER_ID', columns='RULE_NO', values=['THRESHOLD_FLG'])
#col[1] contains only the first name of the column
risk_per_analytical.columns = [col[1] for col in risk_per_analytical.columns.values]
# risk_per_analytical = risk_per_analytical.reset_index()
risk_per_analytical = risk_per_analytical.reset_index()



In [46]:
#weight columns contains all the rule columns
weight_col = risk_per_analytical.columns
weight_col = [col for col in weight_col if col.startswith('P0')]
#Risk_weight contains the weightage of the weight_col
risk_weight = risk_weight[risk_weight['RULE_NO'].isin(weight_col)]['WEIGHTAGE']
risk_weight
# risk_weight.pivot(columns='RULE_NO', values=['WEIGHTAGE'])

69     6
70     6
73     6
74     5
75     4
76     6
78     5
82     6
85     8
86     5
87     5
88     5
90     1
93     1
96     1
107    4
108    5
110    4
111    4
Name: WEIGHTAGE, dtype: int64

In [47]:
#creating the dataframe of the weight_col to make comparison easier
rule_weight_df = pd.DataFrame(data = [risk_weight.values],columns = weight_col)
rule_weight_df

P002001001  P002001002  P002001007  P002001008  P002001009  P002001010  \
0           6           6           6           5           4           6   

   P002001012  P002001018  P002001023  P002001024  P002001025  P002001026  \
0           5           6           8           5           5           5   

   P002001028  P002001031  P002001034  P003001004  P003001007  P003001009  \
0           1           1           1           4           5           4   

   P003001010  
0           4

In [48]:
weighted_avg_denom = pd.read_sql_query("select sum(WEIGHTAGE) from FINCORE_DB.Fincore_Analytics.RISK_LKP_RULES with (nolock) where PERSON_TYPE = 'Individual' and IS_ACTIVE_FLAG = 1 and RISK_TYPE <> 'Auto High Rules ' ",conn)

weighted_avg_denom = weighted_avg_denom.iloc[0,0]
weighted_avg_denom

218

In [49]:
risk_person_summary_stg_rules = pd.read_sql_query("Select * from fincore_db.fincore_analytics.risk_person_summary_stg_rules with (nolock)",conn)

weights = pd.read_sql_query("Select RULE_NO,WEIGHTAGE from fincore_db.fincore_analytics.risk_lkp_rules with (nolock)",conn)

risk_person_summary_stg_rules = risk_person_summary_stg_rules.merge(weights,how = 'inner',on = 'RULE_NO')

risk_person_summary_stg_rules.drop(columns=  ['RULE_VALUE','RISK_SCORE_STATIC','CREATE_DATE'],inplace = True)

risk_person_summary_stg_rules.drop_duplicates(inplace = True)

risk_person_summary_stg_rules.drop(columns= ['RULE_NO'],inplace = True)

denominator = risk_person_summary_stg_rules.groupby('PERSON_MASTER_ID').sum()

risk_per_analytical = risk_per_analytical.merge(denominator,how = 'inner',on = 'PERSON_MASTER_ID')

In [50]:
risk_per_analytical

PERSON_MASTER_ID  P002001001  P002001002  P002001007  P002001008  \
0                   1         NaN         NaN         1.0         NaN   
1                   2         NaN         NaN         1.0         NaN   
2                   3         1.0         1.0         1.0         1.0   
3                   4         1.0         1.0         1.0         1.0   
4                   5         NaN         1.0         1.0         1.0   
..                ...         ...         ...         ...         ...   
206               224         NaN         NaN         1.0         NaN   
207               225         1.0         1.0         1.0         1.0   
208               226         1.0         1.0         1.0         1.0   
209               227         1.0         1.0         1.0         1.0   
210               228         NaN         1.0         1.0         1.0   

     P002001009  P002001010  P002001012  P002001018  P002001023  ...  \
0           NaN         NaN         NaN         NaN         NaN  ...   
1           NaN         NaN         NaN         NaN         NaN  ...   
2           NaN         NaN         1.0         NaN         NaN  ...   
3           NaN         NaN         1.0         NaN         NaN  ...   
4           1.0         1.0         1.0         NaN         NaN  ...   
..          ...         ...         ...         ...         ...  ...   
206         NaN         NaN         NaN         NaN         NaN  ...   
207         1.0         NaN         1.0         NaN         NaN  ...   
208         NaN         NaN         1.0         NaN         NaN  ...   
209         NaN         NaN         1.0         NaN         NaN  ...   
210         1.0         1.0         1.0         NaN         NaN  ...   

     P002001025  P002001026  P002001028  P002001031  P002001034  P003001004  \
0           1.0         NaN         NaN         NaN         NaN         NaN   
1           1.0         NaN         NaN         NaN         NaN         NaN   
2           1.0         NaN         NaN         NaN         NaN         6.0   
3           1.0         NaN         1.0         NaN         NaN         6.0   
4           NaN         NaN         NaN         1.0         NaN         NaN   
..          ...         ...         ...         ...         ...         ...   
206         NaN         NaN         NaN         NaN         NaN         NaN   
207         NaN         NaN         NaN         NaN         NaN         6.0   
208         NaN         NaN         NaN         NaN         NaN         6.0   
209         NaN         NaN         NaN         NaN         NaN         6.0   
210         NaN         NaN         NaN         NaN         NaN         NaN   

     P003001007  P003001009  P003001010  WEIGHTAGE  
0          10.0         NaN         NaN         21  
1          10.0         NaN         NaN         21  
2          10.0         8.0         8.0         55  
3          10.0         8.0         8.0         56  
4          10.0         3.0         4.0         51  
..          ...         ...         ...        ...  
206         NaN         NaN         NaN         11  
207         NaN         NaN         NaN         41  
208         NaN         7.0         7.0         40  
209         NaN         9.0         9.0         40  
210         NaN         7.0         7.0         40  

[211 rows x 21 columns]

In [51]:
# #here calc risk score
# per_analytical_cols = risk_per_analytical.columns.to_list()
# #per_analytical will contain all the columns of risk_per_analytical in addition to below columns
# per_analytical_cols.extend(['RULE_LIST','RISK_SCORE'])
# per_analytical = []
# count = 1
# for ind,row in tqdm(risk_per_analytical.iterrows()):
#     if count%10000 == 0:
#         print(row['PERSON_MASTER_ID'],' HERE')
#     count += 1
#     l1 = []

#     if False: 
# #     ((math.isnan(row['P002001001']) == True) & (math.isnan(row['P002001010']) == True) & (math.isnan(row['P002001011']) == True) & (math.isnan(row['P002001018']) == True & (math.isnan(row['P002001019']) == True) & (math.isnan(row['P002001025']) == True) & (math.isnan(row['P002001026']) == True))) == True: #compare here for nan, in this case risk_score will be null
#         data = row[weight_col]
#         l1.append(row['PERSON_MASTER_ID'])
#         l1.extend(data.values)
#         rule_list = "None" #rule_list will be empty in this case
#         l1.append(rule_list)
#         risk_score = -1 #nan risk score
#         l1.append(risk_score)
# #         print(row)
# #         print(risk_score)
# #         print(rule_list)
# #         l1 containg the entire row including the risk score and appending it with per_analytical list
#         per_analytical.append(l1)
# #         break
#     else:
        
#         data = row[weight_col] * rule_weight_df 
#         l1.append(row['PERSON_MASTER_ID'])
#         l1.extend(data.values[0])
        
#         #Rule_list will include only those columns that has non null values
#         rule_list = [val for val in row.keys() if val.startswith('P0') and ~np.isnan(row[val]) and row[val] > 0 ]
#         rule_list = "_".join(rule_list)
#         num_rules = [val for val in data.values[0] if ~np.isnan(val) and val != 0]
#         l1.append(rule_list)
#         #calculating risk score by summing the row and dividing by count of non null rules
# #         risk_score = (np.sum(num_rules))/(len(num_rules) * 10)
#         risk_score = (np.sum(num_rules))/(weighted_avg_denom)
#         l1.append(risk_score)
#         per_analytical.append(l1)
#         print(l1)
        

In [52]:
risk_per_analytical

PERSON_MASTER_ID  P002001001  P002001002  P002001007  P002001008  \
0                   1         NaN         NaN         1.0         NaN   
1                   2         NaN         NaN         1.0         NaN   
2                   3         1.0         1.0         1.0         1.0   
3                   4         1.0         1.0         1.0         1.0   
4                   5         NaN         1.0         1.0         1.0   
..                ...         ...         ...         ...         ...   
206               224         NaN         NaN         1.0         NaN   
207               225         1.0         1.0         1.0         1.0   
208               226         1.0         1.0         1.0         1.0   
209               227         1.0         1.0         1.0         1.0   
210               228         NaN         1.0         1.0         1.0   

     P002001009  P002001010  P002001012  P002001018  P002001023  ...  \
0           NaN         NaN         NaN         NaN         NaN  ...   
1           NaN         NaN         NaN         NaN         NaN  ...   
2           NaN         NaN         1.0         NaN         NaN  ...   
3           NaN         NaN         1.0         NaN         NaN  ...   
4           1.0         1.0         1.0         NaN         NaN  ...   
..          ...         ...         ...         ...         ...  ...   
206         NaN         NaN         NaN         NaN         NaN  ...   
207         1.0         NaN         1.0         NaN         NaN  ...   
208         NaN         NaN         1.0         NaN         NaN  ...   
209         NaN         NaN         1.0         NaN         NaN  ...   
210         1.0         1.0         1.0         NaN         NaN  ...   

     P002001025  P002001026  P002001028  P002001031  P002001034  P003001004  \
0           1.0         NaN         NaN         NaN         NaN         NaN   
1           1.0         NaN         NaN         NaN         NaN         NaN   
2           1.0         NaN         NaN         NaN         NaN         6.0   
3           1.0         NaN         1.0         NaN         NaN         6.0   
4           NaN         NaN         NaN         1.0         NaN         NaN   
..          ...         ...         ...         ...         ...         ...   
206         NaN         NaN         NaN         NaN         NaN         NaN   
207         NaN         NaN         NaN         NaN         NaN         6.0   
208         NaN         NaN         NaN         NaN         NaN         6.0   
209         NaN         NaN         NaN         NaN         NaN         6.0   
210         NaN         NaN         NaN         NaN         NaN         NaN   

     P003001007  P003001009  P003001010  WEIGHTAGE  
0          10.0         NaN         NaN         21  
1          10.0         NaN         NaN         21  
2          10.0         8.0         8.0         55  
3          10.0         8.0         8.0         56  
4          10.0         3.0         4.0         51  
..          ...         ...         ...        ...  
206         NaN         NaN         NaN         11  
207         NaN         NaN         NaN         41  
208         NaN         7.0         7.0         40  
209         NaN         9.0         9.0         40  
210         NaN         7.0         7.0         40  

[211 rows x 21 columns]

In [53]:
#here calc risk score
per_analytical_cols = risk_per_analytical.columns.to_list()
#per_analytical will contain all the columns of risk_per_analytical in addition to below columns
per_analytical_cols.extend(['RULE_LIST','RISK_SCORE'])
per_analytical_cols.remove('WEIGHTAGE')
per_analytical = []
count = 1
for ind,row in tqdm(risk_per_analytical.iterrows()):
    if count%10000 == 0:
        print(count,' HERE')
    count += 1
    l1 = []

    if False: 
#     ((math.isnan(row['P002001001']) == True) & (math.isnan(row['P002001010']) == True) & (math.isnan(row['P002001011']) == True) & (math.isnan(row['P002001018']) == True & (math.isnan(row['P002001019']) == True) & (math.isnan(row['P002001025']) == True) & (math.isnan(row['P002001026']) == True))) == True: #compare here for nan, in this case risk_score will be null
        data = row[weight_col]
        l1.append(row['PERSON_MASTER_ID'])
        l1.extend(data.values)
        rule_list = "None" #rule_list will be empty in this case
        l1.append(rule_list)
        risk_score = -1 #nan risk score
        l1.append(risk_score)
#         print(row)
#         print(risk_score)
#         print(rule_list)
#         l1 containg the entire row including the risk score and appending it with per_analytical list
        per_analytical.append(l1)
#         break
    else:
        
        data = row[weight_col] * rule_weight_df 
        l1.append(row['PERSON_MASTER_ID'])
        l1.extend(data.values[0])
        
        #Rule_list will include only those columns that has non null values
        rule_list = [val for val in row.keys() if val.startswith('P0') and ~np.isnan(row[val]) and row[val] > 0 ]
        rule_list = "_".join(rule_list)
        num_rules = [val for val in data.values[0] if ~np.isnan(val) and val != 0]
        l1.append(rule_list)
        #calculating risk score by summing the row and dividing by count of non null rules
#         risk_score = (np.sum(num_rules))/(len(num_rules) * 10)
        risk_score = (np.sum(num_rules))/row['WEIGHTAGE']
        l1.append(risk_score)
        per_analytical.append(l1)
#         print(l1)
        

211it [00:00, 1131.22it/s]


In [54]:
data_ind_U = pd.read_sql_query('''select PERSON_MASTER_ID,PEER_GROUP_ID from fincore_db.fincore_analytics.risk_person_summary_stg with (nolock) ''',conn)

In [55]:
#creating a new dataframe
risk_per_analytical_modified = pd.DataFrame(data = per_analytical,columns=per_analytical_cols)

In [56]:
#Merging it with data_ind_U to get the peer_group_id
risk_per_analytical_modified_df = pd.merge(data_ind_U[['PERSON_MASTER_ID','PEER_GROUP_ID']],risk_per_analytical_modified,on = 'PERSON_MASTER_ID')

In [57]:
risk_per_analytical_modified_df.drop_duplicates(inplace=True)

In [58]:
data = risk_per_analytical_modified_df[['PERSON_MASTER_ID','PEER_GROUP_ID','RISK_SCORE','RULE_LIST']]
data.to_csv(r"/data/risk_person.csv")

In [59]:
# data = pd.read_csv(r"/data/risk_person.csv")

In [60]:
try:
    data.drop(columns = ['Unnamed: 0'],axis = 1,inplace= True)
except:
    pass
    

In [61]:
data['RISK_SCORE'].fillna(0,inplace=True)

/opt/conda/lib/python3.6/site-packages/pandas/core/series.py:4535: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [62]:
data['RULE_LIST'].fillna('None',inplace=True)

data.to_csv(r"/data/risk_person_final.csv")

In [63]:
data.shape

(211, 4)

In [64]:
data['RISK_SCORE'] = data['RISK_SCORE'].apply(lambda x: round_func(x)) 

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [65]:
auto_high_df = pd.read_sql_query('''Select PERSON_MASTER_ID from fincore_db.fincore_analytics.RISK_PERSON_SUMMARY_STG_RULES A inner join fincore_db.fincore_analytics.RISK_LKP_RULES  B ON A.RULE_NO =  B.RULE_NO WHERE RISK_TYPE = 'Auto High Rules ' ''',conn)
auto_high = auto_high_df['PERSON_MASTER_ID'].to_list()
data = data[~data['PERSON_MASTER_ID'].isin(auto_high)]

In [66]:
# config = ConfigParser()
# config.read(r'/notebooks/notebooks/config.ini')
# connection_string = config.get('SERVER', 'connection')
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()

In [67]:
# pyodbc.drivers()
# # conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.33.27;PORT=1433;DATABASE=FINCORE_DB;UID=Fincore_Analytics_USR;PWD=F!NCore@123;TDS_Version=7.2;')
# # conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.27.39;PORT=1433;DATABASE=FINCORE_DB;UID=FINCORE_ANALYTICS_USR;PWD=fCrU@t$2!;TDS_Version=7.2;')
# conn = pyodbc.connect('DRIVER=SQL SERVER;SERVER=172.16.27.39;PORT=1433;DATABASE=HDM_FINCORE_DB;UID=FINC_HDM_USR;PWD=#DM@2022;TDS_Version=7.2;')
# cursor = conn.cursor()
# conn = pyodbc.connect(config_data["connection"])
count = 1
#UPDATING RISK_PERSON
# for ind,row in risk_per_analytical_modified_df.iterrows():
for ind,row in tqdm(data.iterrows()):
    if count%10000 == 0:
        print(row['PERSON_MASTER_ID'],' HERE')
        conn.commit()
    count += 1
    conn.execute('''insert into FINCORE_DB.Fincore_Analytics.RISK_PERSON (PERSON_MASTER_ID,PEER_GROUP_ID,RISK_SCORE,CREATE_DATE)
    values ({},'{}',{},'{}',{})'''.format(row['PERSON_MASTER_ID'],row['PEER_GROUP_ID'],row['RISK_SCORE'],'Getdate()'))
    
#     conn.execute('''insert into FINCORE_DB.Fincore_Analytics.RISK_PERSON (PERSON_MASTER_ID,PEER_GROUP_ID,RISK_SCORE,RISK_RULE_LIST)
#     values (?,?,?,?)''',(row['PERSON_MASTER_ID'],row['PEER_GROUP_ID'],row['RISK_SCORE'],row['RULE_LIST']))

conn.commit()
conn.close()

209it [00:00, 950.41it/s]


In [68]:
# config = ConfigParser()
# config.read(r'/notebooks/notebooks/config.ini')
# connection_string = config.get('SERVER', 'connection')
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()

In [69]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [70]:
risk_person =  pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON] with (nolock)",conn)

In [71]:
risk_person

PERSON_MASTER_ID        PEER_GROUP_ID  RISK_SCORE  \
0                   1     Private Sector_0           3   
1                   2     Private Sector_0           3   
2                   3     Private Sector_0           3   
3                   4     Private Sector_0           3   
4                   5     Private Sector_0           2   
..                ...                  ...         ...   
206               224     OTHERS_CHANGED_0           1   
207               225     OTHERS_CHANGED_2           1   
208               226     OTHERS_CHANGED_9           3   
209               227     OTHERS_CHANGED_3           3   
210               228  Government Sector_0           2   

                                        RISK_RULE_LIST             CREATE_DATE  
0          P002001007_P002001024_P002001025_P003001007 2023-03-04 12:40:21.573  
1          P002001007_P002001024_P002001025_P003001007 2023-03-04 12:40:21.573  
2    P002001001_P002001002_P002001007_P002001008_P0... 2023-03-04 12:40:21.573  
3    P002001001_P002001002_P002001007_P002001008_P0... 2023-03-04 12:40:21.577  
4    P002001002_P002001007_P002001008_P002001009_P0... 2023-03-04 12:40:21.577  
..                                                 ...                     ...  
206                              P002001007_P002001024 2023-03-04 12:40:21.787  
207  P002001001_P002001002_P002001007_P002001008_P0... 2023-03-04 12:40:21.787  
208  P002001001_P002001002_P002001007_P002001008_P0... 2023-03-04 12:40:21.787  
209  P002001001_P002001002_P002001007_P002001008_P0... 2023-03-04 12:40:21.790  
210  P002001002_P002001007_P002001008_P002001009_P0... 2023-03-04 12:40:21.790  

[211 rows x 5 columns]

In [72]:
%store -r no_of_records

In [73]:
no_of_records

211

In [74]:
%store -r job_id

job_id

347

In [75]:
# config = ConfigParser()
# config.read(r'/notebooks/notebooks/config.ini')
# connection_string = config.get('SERVER', 'connection')
# conn = pyodbc.connect(connection_string)
# cursor = conn.cursor()


In [76]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [77]:
cursor.execute(f'''Update FINCORE_DB.Fincore_Analytics.AUDIT_INTGN set JOB_END_DATE = Getdate(),JOB_STATUS = 'Success',NO_OF_RECORD = {no_of_records} where  JOB_RUN_ID ={job_id}''')

cursor.commit()
conn.close()